# Retail QA Agent using Granite 3.1 and Docling


# Install Granite3.1 locally

Install and import the necessary libraries and modules.

In [10]:
!ollama pull granite3.1-dense:8b
!ollama pull nomic-embed-text

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling 44d19d212d76... 100% ▕████████████████▏ 5.0 GB                         
pulling f76a906816c4... 100% ▕████████████████▏ 1.4 KB                         
pulling f7b956e70ca3... 100% ▕████████████████▏   69 B                         
pulling 492069a62c25... 100% ▕████████████████▏  11 KB                         
pulling f9cd69f4077d... 100% ▕████████████████▏  491 B                         
verifying sha256 digest 
writing manifest 
success 
pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         
pulling 31df23ea7daa... 100% ▕████████████████▏  420 B                         
verifying sha256 digest 
writing manifest 
success 


In [11]:
# Install required packages
!pip install -q "langchain>=0.1.0" "langchain-community>=0.0.13" "langchain-core>=0.1.17" \
    "langchain-ollama>=0.0.1" "pdfminer.six>=20221105" "markdown>=3.5.2" "docling>=2.0.0" \
    "beautifulsoup4>=4.12.0" "unstructured>=0.12.0" "chromadb>=0.4.22" "faiss-cpu>=1.7.4" \
    "requests>=2.32.0"


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip


In [12]:
# Required imports
import os
import tempfile
import shutil
from pathlib import Path
from IPython.display import Markdown, display

# Docling imports
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions, TesseractCliOcrOptions
from docling.document_converter import DocumentConverter, PdfFormatOption, WordFormatOption, SimplePipeline

# LangChain imports
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings, OllamaLLM
from langchain_community.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

### Step 2. Document format detection


In [13]:
def get_document_format(file_path) -> InputFormat:
    """Determine the document format based on file extension"""
    try:
        file_path = str(file_path)
        extension = os.path.splitext(file_path)[1].lower()

        format_map = {
            '.pdf': InputFormat.PDF,
            '.docx': InputFormat.DOCX,
            '.doc': InputFormat.DOCX,
            '.pptx': InputFormat.PPTX,
            '.html': InputFormat.HTML,
            '.htm': InputFormat.HTML
        }
        return format_map.get(extension, None)
    except:
        return "Error in get_document_format: {str(e)}"

### Step 3. Document conversion

In [14]:
def convert_document_to_markdown(doc_path) -> str:
    """Convert document to markdown using simplified pipeline"""
    try:
        # Convert to absolute path string
        input_path = os.path.abspath(str(doc_path))
        print(f"Converting document: {doc_path}")

        # Create temporary directory for processing
        with tempfile.TemporaryDirectory() as temp_dir:
            # Copy input file to temp directory
            temp_input = os.path.join(temp_dir, os.path.basename(input_path))
            shutil.copy2(input_path, temp_input)

            # Configure pipeline options
            pipeline_options = PdfPipelineOptions()
            pipeline_options.do_ocr = False  # Disable OCR temporarily
            pipeline_options.do_table_structure = True

            # Create converter with minimal options
            converter = DocumentConverter(
                allowed_formats=[
                    InputFormat.PDF,
                    InputFormat.DOCX,
                    InputFormat.HTML,
                    InputFormat.PPTX,
                ],
                format_options={
                    InputFormat.PDF: PdfFormatOption(
                        pipeline_options=pipeline_options,
                    ),
                    InputFormat.DOCX: WordFormatOption(
                        pipeline_cls=SimplePipeline
                    )
                }
            )

            # Convert document
            print("Starting conversion...")
            conv_result = converter.convert(temp_input)

            if not conv_result or not conv_result.document:
                raise ValueError(f"Failed to convert document: {doc_path}")

            # Export to markdown
            print("Exporting to markdown...")
            md = conv_result.document.export_to_markdown()

            # Create output path
            output_dir = os.path.dirname(input_path)
            base_name = os.path.splitext(os.path.basename(input_path))[0]
            md_path = os.path.join(output_dir, f"{base_name}_converted.md")

            # Write markdown file
            print(f"Writing markdown to: {base_name}_converted.md")
            with open(md_path, "w", encoding="utf-8") as fp:
                fp.write(md)

            return md_path
    except:
        return f"Error converting document: {doc_path}"

### Step 4. QA chain setup

In [15]:
def setup_qa_chain(markdown_path: Path, embeddings_model_name:str = "nomic-embed-text:latest", model_name: str = "granite3.1-dense:8b"):
    """Set up the QA chain for document processing"""
    # Load and split the document
    loader = UnstructuredMarkdownLoader(str(markdown_path)) 
    documents = loader.load()
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50,
        length_function=len
    )
    texts = text_splitter.split_documents(documents)
    # texts= documents
    
    # Create embeddings and vector store
    embeddings = OllamaEmbeddings(
        model=embeddings_model_name
        )
    vectorstore = FAISS.from_documents(texts, embeddings)
    
    # Initialize LLM
    llm = OllamaLLM(
        model=model_name,
        temperature=0
    )
    
    # Set up conversation memory
    memory = ConversationBufferMemory(
        memory_key="chat_history",
        output_key="answer",
        return_messages=True
    )
    
    # Create the chain
    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(
            search_kwargs={"k": 10}
            ),
        memory=memory,
        return_source_documents=True
    )
    
    return qa_chain

### Step 5. Set up question-answering interface


In [16]:
def ask_question(qa_chain, question: str):
    """Ask a question and display the answer"""
    result = qa_chain.invoke({"question": question})
    display(Markdown(f"**Question:** {question}\n\n**Answer:** {result['answer']}"))

### Step 6. Ask Questions to the Document

In [20]:
# Process a document
doc_path = Path("l0.pdf")  # Replace with your document path

# Check format and process
doc_format = get_document_format(doc_path)
if doc_format:
    md_path = convert_document_to_markdown(doc_path)
    qa_chain = setup_qa_chain(md_path)
    
    # Example questions
    questions = [
        "What is the main topic of this document?",
        "What are the key points discussed?",
        "Can you summarize the conclusions?",
    ]
    
    for question in questions:
        ask_question(qa_chain, question)
else:
    print(f"Unsupported document format: {doc_path.suffix}")

Converting document: l0.pdf
Starting conversion...
Exporting to markdown...
Writing markdown to: l0_converted.md


**Question:** What is the main topic of this document?

**Answer:** The main topic of this document is a legal request related to an insurance policy. Angela Berry, an attorney, has been retained by Courtney Sosa to handle the estate of Lukas Juarez. The letter informs Budget Mutual Insurance Company that they represent Courtney Sosa and are requesting information about a life insurance policy (#951033310) held by Lukas Juarez with their company. The attorney also asks for confirmation of receipt of the representation letter and instructs the insurance company not to contact their client directly.

**Question:** What are the key points discussed?

**Answer:** The main details outlined in this legal request are as follows:

1. The sender, Angela Berry, is an attorney representing Courtney Sosa in the estate of Lukas Juarez.
2. The deceased individual, Lukas Juarez, had a life insurance policy with Budget Mutual Insurance Company, identified by the number #951033310.
3. The attorney is requesting that the full policy amount of $50,000 be forwarded to her office.
4. She also asks for an acknowledgement of their demand and requests umbrella policy information if applicable.
5. The attorney's secretary should receive any information regarding liens on the policy.
6. The sender has instructed that the insurance company should not contact their client (Courtney Sosa) going forward.
7. The letter serves as a Notice of Representation, indicating that the law firm is now handling the estate's matters related to this insurance policy.

**Question:** Can you summarize the conclusions?

**Answer:** Angela Berry, an attorney representing Courtney Sosa, has written a letter to Budget Mutual Insurance Company regarding Lukas Juarez's life insurance policy (#951033310). The key points of her request are:

1. Acknowledgment of Representation: She requests that the insurance company acknowledge receipt of their letter of representation, indicating they have been informed about her role as the legal representative for Courtney Sosa.

2. Policy Amount and Forwarding: Angela Berry asks for the full policy amount of $50,000 to be forwarded to her office. This suggests that Lukas Juarez's life insurance policy has a death benefit of $50,000.

3. Acknowledgement of Demand: She requests an acknowledgement from the insurance company regarding their demand for the full policy amount.

4. Umbrella Policy Information: If applicable, she asks for information about any umbrella policies that might be linked to Lukas Juarez's life insurance policy.

5. Liens Information: She requests any information related to liens on Lukas Juarez's policy. This could indicate potential debts or claims against the policy before distribution to the beneficiary, Courtney Sosa.

6. Communication Instructions: Angela Berry instructs the insurance company not to contact their client (Courtney Sosa) directly regarding this matter, implying that all communication should go through her office.

In summary, Angela Berry is requesting the full death benefit from Lukas Juarez's life insurance policy, along with related information and documentation, while instructing the insurance company to communicate only with her office concerning this matter.


## Conclusion

Using Docling and Granite 3.1, we have built a question answering system for Retail Documents